In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import category_encoders as ce

In [2]:
og_train = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv", index_col="id")
og_test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv", index_col="id")

In [3]:
engine_re = r'(?:(\d+\.?\d*)HP)? ?(?:(\d+\.?\d*)(?:L| Liter))? ?((?:Straight)? ?(?:V\d+ ?(?:Cylinder)?|\d+ Cylinder|I\d+|H\d+|Flat ?\d+|W\d+))? ?(?:(\d+)V)? ?(DDI|GDI|PFI|MPFI|SIDI|PDI)? ?(OHV|DOHC|SOHC)? ?(Turbo|Twin Turbo|Supercharged|TFSI|T\/C|TSI)? ?(?:Engine)? ?(?:(\d+\.?\d*) ?A[hH])?[ \/]*(?:FR ?(\d+\.?\d*)kW)?[ \/]*(?:RR ?(\d+\.?\d*)kW)?[ \/]?(?:\((\d+)V?\))?(.*)'

In [4]:
def extract_engine_features(df): 
    extracted_features = ["HP","L","Cylinders","Valves","Fuel_injection","Valve_train_design","Forced_induction","Ah","FRKW","RRKW","Volts","Engine_type"]
    df = df.copy()
    df_extracted = df["engine"].str.extract(engine_re)
    df_extracted.columns = extracted_features
    
    #Clean value types
    float_feat = ["HP","L","Valves","Ah","FRKW","RRKW","Volts"]
    for feat in float_feat: 
        df_extracted[feat] = df_extracted[feat].astype(float)
    
    df = df.join(df_extracted)
    
    df = df.drop("engine", axis=1)
    
    return df

In [5]:
def fe_accident(df): 
    accident_map = {
        "None reported": 0,
        "At least 1 accident or damage reported": 1
    }
    df = df.copy()
    
    # more than 50% of vehicules with a milage > 175k have accidents
    df.loc[(df.accident.isna()) & (df.milage > 175_000), "accident"] = "At least 1 accident or damage reported"
    df["accident"] = df["accident"].fillna("None reported")
    df["accident"] = df["accident"].map(accident_map)
    
    return df

In [6]:
def fe_clean_title(df):
    df = df.copy()
    df["clean_title"] = df["clean_title"].fillna("No")
    df["clean_title"] = df["clean_title"].map({"No": 0, "Yes": 1})
    
    return df

In [7]:
def feature_engineering(train, test): 
    fe_train = train.copy()
    fe_test = test.copy()
    
    fe_train = extract_engine_features(fe_train)
    fe_test = extract_engine_features(fe_test)


    fe_train = fe_accident(fe_train)
    fe_test = fe_accident(fe_test)
    
    fe_train = fe_clean_title(fe_train)
    fe_test = fe_clean_title(fe_test)
    
    
    
    return fe_train, fe_test
    

In [8]:
train, test = feature_engineering(og_train, og_test)

In [9]:
unused_cols = ["price"]
ordinal_cols = ["model","ext_col","int_col"]
num_cols_mean = ["model_year","milage", "HP"]
num_cols_zero = ["L","Valves","Ah","FRKW","RRKW","Volts"]
target_encode_cols = []
one_hot_cols = ["fuel_type","transmission","Cylinders","Fuel_injection","Valve_train_design","Forced_induction","Engine_type"]

In [10]:
num_mean_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))
])
num_zero_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value=0))
])
# So we keep an order in the brand encoding
mean_brand_prices = train.groupby("brand")["price"].mean()
brand_sorted = mean_brand_prices.sort_values(ascending=True)
brand_sorted = brand_sorted.reset_index()
sorted_brands = brand_sorted["brand"].tolist()

brand_pipeline = Pipeline(steps=[
    ("encoder", OrdinalEncoder(categories=[sorted_brands]))
])

cat_ord_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder_ord", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])
cat_one_hot_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder_onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
target_encode_pipeline = Pipeline(steps=[
    ("target_encoder", ce.TargetEncoder())
])

passthrough = False 
if passthrough: 
    preprocessor = ColumnTransformer(transformers=[
        ("num_mean", num_mean_pipeline, num_cols_mean),
        ("num_zero", num_zero_pipeline, num_cols_zero),
        ("target_enc", target_encode_pipeline, target_encode_cols),
        ("cat_ord", cat_ord_pipeline, ordinal_cols),
        ("cat_onehot", cat_one_hot_pipeline, one_hot_cols),
        ("brand",brand_pipeline, ["brand"]),
        ("passthrough", "passthrough", unused_cols)
    ])
else: 
    preprocessor = ColumnTransformer(transformers=[
        ("num_mean", num_mean_pipeline, num_cols_mean),
        ("num_zero", num_zero_pipeline, num_cols_zero),
        ("target_enc", target_encode_pipeline, target_encode_cols),
        ("cat_ord", cat_ord_pipeline, ordinal_cols),
        ("cat_onehot", cat_one_hot_pipeline, one_hot_cols),
        ("brand",brand_pipeline, ["brand"])
    ])

In [11]:
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import HuberRegressor, RidgeCV, BayesianRidge, Ridge, LinearRegression, LassoCV, ElasticNetCV, LassoLarsCV, LassoLarsIC, LarsCV, Lars, SGDRegressor, RANSACRegressor, ElasticNet, Lasso, OrthogonalMatchingPursuitCV, PassiveAggressiveRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.dummy import DummyRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import OrthogonalMatchingPursuit
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score

In [12]:
X = train.drop("price", axis=1)
y = train["price"]

In [13]:
model = HistGradientBoostingRegressor(
    learning_rate=0.038,
    max_iter=1389,
    max_leaf_nodes=765,
    max_depth=430,
    min_samples_leaf=100,
    l2_regularization=0.33,
)

In [14]:
pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])

In [15]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_mean',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['model_year', 'milage',
                                                   'HP']),
                                                 ('num_zero',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['L', 'Valves', 'Ah', 'FRKW',
                                                   'RRKW', 'Volts']),
                                                 ('target_enc',
                                                  Pipeline(steps=[('target_encoder',
                                                                   TargetEncoder())]...
                                                                                               'Mercury',
                                                                                               'Volkswagen',
                                                                                               'Lotus',
                                                                                               'Chrysler',
                                                                                               'Plymouth',
                                                                                               'Saab',
                                                                                               'Acura',
                                                                                               'Toyota',
                                                                                               'Nissan',
                                                                                               'Buick',
                                                                                               'Suzuki',
                                                                                               'Lincoln',
                                                                                               'Volvo',
                                                                                               'FIAT',
                                                                                               'Dodge',
                                                                                               'INFINITI',
                                                                                               'Jeep',
                                                                                               'smart',
                                                                                               'Lexus',
                                                                                               'Karma', ...]]))]),
                                                  ['brand'])])),
                ('model',
                 HistGradientBoostingRegressor(l2_regularization=0.33,
                                               learning_rate=0.038,
                                               max_depth=430, max_iter=1389,
                                               max_leaf_nodes=765,
                                               min_samples_leaf=100))])

In [16]:
preds = pipeline.predict(test)
preds

array([18368.06386922, 78004.76315954, 56024.59999205, ...,
       22880.25366778, 15389.97668443, 35649.4313783 ])

In [17]:
submission_file = test.reset_index()[['id']]
submission_file['price'] = preds
submission_file = submission_file.set_index("id")
submission_file.to_csv("/kaggle/working/submission.csv")